<h1>BPE 예제<h1>

In [1]:
import re, collections

vocab = {'l o w </w>' : 5,
         'l o w e r </w>' : 2,
         'n e w e s t </w>':6,
         'w i d e s t </w>':3
         }

def get_stats(vocab):
    # 개수를 세는 모듈
    pairs = collections.defaultdict(int)

    for word, freq in vocab.items():
        symbols = word.split()

        for i in range(len(symbols)-1):
            # 연속된 두 글자씩의 빈도수 저장
            pairs[symbols[i],symbols[i+1]] += freq
            # print(pairs[symbols[i],symbols[i+1]])
    
    return pairs

get_stats(vocab)


defaultdict(int,
            {('l', 'o'): 7,
             ('o', 'w'): 7,
             ('w', '</w>'): 5,
             ('w', 'e'): 8,
             ('e', 'r'): 2,
             ('r', '</w>'): 2,
             ('n', 'e'): 6,
             ('e', 'w'): 6,
             ('e', 's'): 9,
             ('s', 't'): 9,
             ('t', '</w>'): 9,
             ('w', 'i'): 3,
             ('i', 'd'): 3,
             ('d', 'e'): 3})

In [2]:
def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')

    for word in v_in:
        # pair 두 글자를 붙임 (e s -> es)
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    
    return v_out

In [3]:
# 특수문자에 \추가하여 정규표현식에 사용
# re.escape 함수는 문자열을 입력받으면 특수문자들을 이스케이프 처리시켜 준다.
re.escape(' '.join(('e', 's')))

'e\\ s'

In [4]:
vocab

{'l o w </w>': 5,
 'l o w e r </w>': 2,
 'n e w e s t </w>': 6,
 'w i d e s t </w>': 3}

In [5]:
merge_vocab(('e', 's'), vocab)

{'l o w </w>': 5,
 'l o w e r </w>': 2,
 'n e w es t </w>': 6,
 'w i d es t </w>': 3}

In [6]:
num_merges = 10

for i in range(num_merges):
    pairs = get_stats(vocab)
    best = max(pairs, key=pairs.get)
    vocab = merge_vocab(best, vocab)
    print(best)

('e', 's')
('es', 't')
('est', '</w>')
('l', 'o')
('lo', 'w')
('n', 'e')
('ne', 'w')
('new', 'est</w>')
('low', '</w>')
('w', 'i')


In [7]:
vocab

{'low</w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'wi d est</w>': 3}

<br>
<br>
<br>
<h1>SentencePiece 예제<h1>

In [8]:
!pip install sentencepiece

In [9]:
import sentencepiece as spm

input_file = 'spm_input.txt'

corpus = [
    '딥러닝은 인공지능의 한 종류다',
    '딥러닝에는 지도학습과 비지도학습이 있다',
    '인공지능은 어렵다',
    '딥러닝도 어렵다',
    '인공지능은 신기하다'
]

with open(input_file, 'w', encoding='utf-8') as f:
    for sent in corpus:
        f.write('{}\n'.format(sent))

templates = '--input={} --model_prefix={} --vocab_size={}'

vocab_size = 30
prefix = 'data'
cmd = templates.format(input_file, prefix, vocab_size)

spm.SentencePieceTrainer.Train(cmd)

In [10]:
sp = spm.SentencePieceProcessor()
sp.Load('{}.model'.format(prefix))

sp.EncodeAsPieces('딥러닝은 인공지능의 한 종류다')

['▁딥러닝', '은', '▁', '인', '공', '지', '능', '의', '▁', '한', '▁', '종', '류', '다']

In [11]:
sp.EncodeAsIds('딥러닝은 인공지능의 한 종류다')

[4, 7, 3, 9, 14, 28, 13, 24, 3, 29, 3, 27, 21, 6]

In [12]:
with open('{}.vocab'.format(prefix), encoding='utf-8') as f:
    vocabs = [doc.strip() for doc in f]

print('num of vocabs = {}'.format(len(vocabs)))

num of vocabs = 30


In [13]:
vocabs # 숫자가 낮을수록 빈도수 높은 것

['<unk>\t0',
 '<s>\t0',
 '</s>\t0',
 '▁\t-1.84384',
 '▁딥러닝\t-2.42717',
 '▁어렵다\t-2.92717',
 '다\t-2.92717',
 '은\t-2.92717',
 '학\t-3.92617',
 '인\t-3.92627',
 '어\t-3.92637',
 '습\t-3.92647',
 '렵\t-3.92657',
 '능\t-3.92667',
 '공\t-3.92677',
 '러\t-3.92687',
 '딥\t-3.92697',
 '닝\t-3.92707',
 '과\t-3.92717',
 '는\t-3.92717',
 '도\t-3.92717',
 '류\t-3.92717',
 '비\t-3.92717',
 '에\t-3.92717',
 '의\t-3.92717',
 '이\t-3.92717',
 '있\t-3.92717',
 '종\t-3.92717',
 '지\t-3.92717',
 '한\t-3.92717']